### Bibliotheques

In [ ]:
#Importation des bibliotheques necessaires
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
import matplotlib.pyplot as plt

import os
from distutils.dir_util import copy_tree, remove_tree

from PIL import Image
from random import randint

from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn.metrics import matthews_corrcoef as MCC
from sklearn.metrics import balanced_accuracy_score as BAS
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC

from joblib import dump, load

import tensorflow_addons as tfa
from keras.utils.vis_utils import plot_model
from tensorflow.keras import Sequential, Input
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.layers import Conv2D, Flatten
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg19 import VGG19

from tensorflow.keras.preprocessing.image import ImageDataGenerator as IDG
from tensorflow.keras.layers import SeparableConv2D, BatchNormalization, GlobalAveragePooling2D


print("TensorFlow Version:", tf.__version__)

### Data Pre-Processing

In [ ]:
#Dossiers des dataset

base_dir = "../dataset/"
root_dir = "./"
work_dir = root_dir + "adni/"
#work_dir = root_dir + "kaggle/"
    
print("Working Directory Contents:", os.listdir(work_dir))

In [ ]:
#Initialisation des classes et la taille des images
WORK_DIR = './adni/'

CLASSES = ['AD','MCI','CN']

IMG_SIZE = 176
IMAGE_SIZE = [176, 176]
DIM = (IMG_SIZE, IMG_SIZE)

In [ ]:
#Augmentation des données

ZOOM = [.99, 1.01]
BRIGHT_RANGE = [0.8, 1.2]
HORZ_FLIP = False
FILL_MODE = "constant"
DATA_FORMAT = "channels_last"

work_dr = IDG(rescale = 1./255, brightness_range=BRIGHT_RANGE, zoom_range=ZOOM, data_format=DATA_FORMAT, fill_mode=FILL_MODE, horizontal_flip=HORZ_FLIP)

train_data_gen = work_dr.flow_from_directory(directory=WORK_DIR, target_size=DIM, batch_size=6000, shuffle=True)

In [ ]:
#Normalisation
image_pixels = train_data.reshape(-1, train_data.shape[-1])

# Creation MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(image_pixels)

# Transformer les pixels en [0, 1]
image_pixels = scaler.transform(image_pixels)

# Retoure au demention originale des images
train_data = image_pixels.reshape(train_data.shape)

In [ ]:
#Affichage des images irm
"""def show_images(generator,y_pred=None):
    
    Input: An image generator,predicted labels (optional)
    Output: Displays a grid of 9 images with lables
    
    
    # get image lables
    labels =dict(zip([0,1,2,3], CLASSES))
    
    # get a batch of images
    x,y = generator.next()
    
    # display a grid of 9 images
    plt.figure(figsize=(10, 10))
    if y_pred is None:
        for i in range(9):
            ax = plt.subplot(3, 3, i + 1)
            idx = randint(0, 6400)
            plt.imshow(x[idx])
            plt.axis("off")
            plt.title("Class:{}".format(labels[np.argmax(y[idx])]))
                                                     
    else:
        for i in range(9):
            ax = plt.subplot(3, 3, i + 1)
            plt.imshow(x[i])
            plt.axis("off")
            plt.title("Actual:{} \nPredicted:{}".format(labels[np.argmax(y[i])],labels[y_pred[i]]))
    
# Display Train Images
show_images(train_data_gen)"""

In [ ]:
#Récupérer les données à partir de l'itérateur ImageDataGenerator.

train_data, train_labels = train_data_gen.next()

In [ ]:
#Les dimensions des données

print(train_data.shape, train_labels.shape)

In [ ]:
#Effectuer une sur-échantillonnage (OverSampling) des données, car les classes sont déséquilibrées.

sm = SMOTE(random_state=42)

train_data, train_labels = sm.fit_resample(train_data.reshape(-1, IMG_SIZE * IMG_SIZE * 3), train_labels)

train_data = train_data.reshape(-1, IMG_SIZE, IMG_SIZE, 3)

print(train_data.shape, train_labels.shape)

### Extraction des caracterestiques

In [ ]:
#Importer le modele pre entraine VGG19 (import weights)
vgg = VGG19(input_shape=(176, 176, 3), include_top=False, weights="imagenet")

In [ ]:
#Importer le modele pre entraine InceptionV3 (import weights)
inception = InceptionV3(input_shape=(176, 176, 3), include_top=False, weights="imagenet")

In [ ]:
#extraction des caracterestiques depuis VGG19
batch_size = 32
vgg_features = []

for i in range(0, len(train_data), batch_size):
  batch = train_data[i:i+batch_size]  
  batch_features = vgg.predict(batch)
  vgg_features.append(batch_features)

vgg_features = np.concatenate(vgg_features, axis=0)

In [ ]:
#extraction des caracterestiques depuis InceptionV3
batch_size = 32
inception_features = []

for i in range(0, len(train_data), batch_size):
  batch = train_data[i:i+batch_size]  
  batch_features = inception.predict(batch)
  inception_features.append(batch_features)

inception_features = np.concatenate(inception_features, axis=0)

In [ ]:
#Redimentionner les caracterestiques de VGG19 en vecteur 2D
vgg_features=vgg_features.reshape(vgg_features.shape[0], -1)

In [ ]:
#Redimentionner les caracterestiques de InceptionV3 en vecteur 2D
inception_features=inception_features.reshape(vgg_features.shape[0], -1)

### Concatination et Splittng

In [ ]:
#La concatination des caracterestiques
features = np.concatenate([vgg_features, inception_features], axis=1)

In [ ]:
#Affectaion des caracterestiques a nouvelle variable pour passer au SVM
X = features

In [ ]:
#Affectation des labels
Y=train_labels

In [ ]:
##Diviser les données en ensembles d'entraînement et de test.
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

### Entrainement de svm

In [ ]:
#Le modele SVM
svm = SVC(kernel='linear', C=0.6, random_state=42)

In [ ]:
#Entrainement de SVM
svm.fit(X_train, np.argmax(Y_train, axis=1))

### Test et resultats

In [ ]:
#Prediction sur les données de test
Y_pred = svm.predict(X_test)

In [ ]:
#Calcule de Accuracy
accuracy = accuracy_score(np.argmax(Y_test, axis=1), Y_pred)
print('Accuracy:', accuracy)

In [ ]:
#Redimentionnemnt en matrice
n_classes = len(np.unique(Y_pred))
Y_pred = np.eye(n_classes)[Y_pred]

In [ ]:
#Matrice de confusion
pred_ls = np.argmax(Y_pred, axis=1)
test_ls = np.argmax(Y_test, axis=1)
conf_matrix = confusion_matrix(test_ls, pred_ls)

# Normalize 
conf_matrix_norm = conf_matrix / conf_matrix.sum(axis=1)[:,np.newaxis] 

plt.figure(figsize=(8, 6), dpi=80, facecolor='w', edgecolor='k')
ax = sns.heatmap(conf_matrix_norm, 
                cmap='Blues', 
                annot=True, 
                fmt='.2%',
                xticklabels=CLASSES,
                yticklabels=CLASSES)

#plt.title('Normalized Confusion Matrix')
plt.xlabel('Classe prédite')
plt.ylabel('Classe réelle')
plt.show()

In [ ]:
#Le rapport de classification : precision, reccall, f1 score

def roundoff(arr):
    """To round off according to the argmax of each predicted label array. """
    arr[np.argwhere(arr != arr.max())] = 0
    arr[np.argwhere(arr == arr.max())] = 1
    return arr

for labels in Y_pred:
    labels = roundoff(labels)

print(classification_report(Y_test, Y_pred, target_names=CLASSES))

### Sauvgarde des modeles

In [ ]:
#Sauvgarde de modeles VGG19 et InceptionV3 pour extraction des caracterestiques

#VGG19
vgg_model_dir = work_dir + "vgg_model"
vgg.save(vgg_model_dir, save_format='h5')
os.listdir(work_dir)

#InceptionV3
inception_model_dir = work_dir + "inception_model"
inception.save(inception_model_dir, save_format='h5')
os.listdir(work_dir)

In [ ]:
#Sauvgarde de modele SVM
model_folder = '../Saved models/'
model_filename = 'svm_model.sav'
model_path = os.path.join(model_folder, model_filename)
dump(svm, model_path)